<a href="https://colab.research.google.com/github/ArkS0001/AI-Agent-Atom-of-Thought-2026/blob/main/granite3_3_agent_multi_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install torch torchvision torchaudio accelerate transformers

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
import torch

model_path="ibm-granite/granite-3.3-2b-instruct"
device="cuda"
model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map=device,
        torch_dtype=torch.bfloat16,
    )
tokenizer = AutoTokenizer.from_pretrained(
        model_path
)

conv = [{"role": "user", "content":"Take a current environmental issue and work backward to devise an innovative prevention strategy that could have averted the problem."}]

input_ids = tokenizer.apply_chat_template(conv, return_tensors="pt", thinking=True, return_dict=True, add_generation_prompt=True).to(device)

set_seed(42)
output = model.generate(
    **input_ids,
    max_new_tokens=8192,
)

prediction = tokenizer.decode(output[0, input_ids["input_ids"].shape[1]:], skip_special_tokens=True)
print(prediction)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/787 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/29.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.48M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

<think>To address this task, I'll choose the environmental issue of plastic pollution in our oceans as a current problem. I'll then work backward to devise an innovative prevention strategy that could have mitigated this issue.

### Current Environmental Issue: Plastic Pollution in Oceans

**Problem Description:**
- Millions of tons of plastic waste enter oceans annually, causing severe harm to marine life, ecosystems, and human health.
- Plastic debris persists in the environment for centuries, breaking down into microplastics that contaminate food chains.

### Working Backward to Develop a Prevention Strategy

1. **Identify Root Causes:**
   - **Consumer Behavior:** Over-reliance on single-use plastics (e.g., bags, straws, bottles).
   - **Industrial Practices:** Lack of sustainable packaging solutions and inadequate waste management systems.
   - **Government Policies:** Insufficient regulations and enforcement regarding plastic production and disposal.

2. **Develop a Prevention St

## AGENT

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch, re, json, random

# ──────────────────────────────────────────
# 1. Load Granite-3.3-2B-Instruct
# ──────────────────────────────────────────
model_name = "ibm-granite/granite-3.3-2b-instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
model.eval()

# ──────────────────────────────────────────
# 2. Define Available Tools
# ──────────────────────────────────────────
TOOLS = [
    {
        "name": "calculator",
        "description": "Evaluate a basic arithmetic expression",
        "arguments": {
            "expression": {
                "type": "string",
                "description": "Arithmetic using digits and + - * / ( )"
            }
        }
    },
    {
        "name": "get_current_weather",
        "description": "Fetch current weather for a location",
        "arguments": {
            "location": {
                "type": "string",
                "description": "City name, e.g. 'Paris, FR'"
            }
        }
    }
]

def calculator_tool(expr: str) -> str:
    if not re.fullmatch(r"[0-9+\-*/ ().]+", expr):
        return "Invalid expression"
    try:
        return str(eval(expr))
    except:
        return "Error"

def get_current_weather(loc: str) -> str:
    temp = random.randint(-5, 35)
    return f"The current temperature in {loc} is {temp}°C."

# ──────────────────────────────────────────
# 3. Run One Tool-Calling Turn
# ──────────────────────────────────────────
def run_with_tools(question: str) -> str:
    conv = [{"role": "user", "content": question}]

    # Initial pass: model may emit a tool_call
    inputs = tokenizer.apply_chat_template(
        conv,
        tools=TOOLS,
        thinking=True,
        return_tensors="pt",
        return_dict=True,
        add_generation_prompt=True
    ).to(device)

    out = model.generate(**inputs, max_new_tokens=512)
    chunk = tokenizer.decode(
        out[0, inputs["input_ids"].shape[1]:],
        skip_special_tokens=False
    )

    # Look for the <|tool_call|> JSON payload
    match = re.search(r"<\|tool_call\|>(\{.*?\})", chunk, re.DOTALL)
    if not match:
        return chunk.strip()  # No tool call

    # Parse the JSON and execute the tool
    call = json.loads(match.group(1))
    name = call["name"]
    args = call["arguments"]

    if name == "calculator":
        result = calculator_tool(args["expression"])
    elif name == "get_current_weather":
        result = get_current_weather(args["location"])
    else:
        result = f"Error: unknown tool {name}"

    # Append the model’s reasoning and tool response
    reasoning = chunk.split("<|tool_call|>")[0]
    conv.append({"role": "assistant", "content": reasoning})
    conv.append({
        "role": "tool_response",
        "name": name,
        "content": json.dumps({"result": result})
    })

    # Final pass: ask Granite to finish its answer
    final_inputs = tokenizer.apply_chat_template(
        conv,
        tools=TOOLS,
        thinking=True,
        return_tensors="pt",
        return_dict=True,
        add_generation_prompt=True
    ).to(device)

    final_out = model.generate(**final_inputs, max_new_tokens=256)
    final_text = tokenizer.decode(
        final_out[0, final_inputs["input_ids"].shape[1]:],
        skip_special_tokens=True
    )
    return final_text.strip()

# ──────────────────────────────────────────
# 4. Example
# ──────────────────────────────────────────
if __name__ == "__main__":
    q = (
        "What is 47 * 13 minus 5? "
        "Also, tell me the current weather in Paris."
    )
    print("Answer:\n", run_with_tools(q))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Answer:
 <|tool_call|>[{"name": "calculator", "arguments": {"expression": "47 * 13 minus 5"}}, {"name": "get_current_weather", "arguments": {"location": "Paris"}}]<|end_of_text|>


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch, re, json, random

# ── 1. Load Granite-3.3-2B-Instruct ─────────────────────────────
model_name = "ibm-granite/granite-3.3-2b-instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map="auto", torch_dtype=torch.bfloat16
)
model.eval()

# ── 2. Define Tools ────────────────────────────────────────────
TOOLS = [
    {
        "name": "calculator",
        "description": "Evaluate a basic arithmetic expression",
        "arguments": {
            "expression": {"type": "string", "description": "Digits and + - * / ( ) only"}
        }
    },
    {
        "name": "get_current_weather",
        "description": "Fetch current weather for a location",
        "arguments": {
            "location": {"type": "string", "description": "City, e.g. 'Paris, FR'"}
        }
    }
]

def calculator_tool(expr: str) -> str:
    if not re.fullmatch(r"[0-9+\-*/ ().]+", expr):
        return "Invalid expression"
    try:
        return str(eval(expr))
    except:
        return "Error"

def get_current_weather(loc: str) -> str:
    temp = random.randint(-5, 35)
    return f"The current temperature in {loc} is {temp}°C."

# ── 3. Single Turn with Multiple Tools ──────────────────────────
def run_with_multiple_tools(question: str) -> str:
    conv = [{"role": "user", "content": question}]

    # Initial generation: model may emit one <|tool_call|> with a JSON array
    inputs = tokenizer.apply_chat_template(
        conv, tools=TOOLS, thinking=True,
        return_tensors="pt", return_dict=True,
        add_generation_prompt=True
    ).to(device)

    out = model.generate(**inputs, max_new_tokens=512)
    chunk = tokenizer.decode(
        out[0, inputs["input_ids"].shape[1]:],
        skip_special_tokens=False
    )

    # Extract array of tool calls
    match = re.search(r"<\|tool_call\|>(\[\{.*?\}\])", chunk, re.DOTALL)
    if not match:
        return chunk.strip()

    calls = json.loads(match.group(1))  # List of {name, arguments}

    # Split out the model's internal reasoning before the call marker
    reasoning = chunk.split("<|tool_call|>")[0]
    conv.append({"role": "assistant", "content": reasoning})

    # Execute each tool and append its response
    for call in calls:
        name, args = call["name"], call["arguments"]
        if name == "calculator":
            result = calculator_tool(args["expression"])
        elif name == "get_current_weather":
            result = get_current_weather(args["location"])
        else:
            result = f"Error: unknown tool {name}"
        conv.append({
            "role": "tool_response",
            "name": name,
            "content": json.dumps({"result": result})
        })

    # Final generation: model sees all tool responses and completes the answer
    final_inputs = tokenizer.apply_chat_template(
        conv, tools=TOOLS, thinking=True,
        return_tensors="pt", return_dict=True,
        add_generation_prompt=True
    ).to(device)

    final_out = model.generate(**final_inputs, max_new_tokens=256)
    final_text = tokenizer.decode(
        final_out[0, final_inputs["input_ids"].shape[1]:],
        skip_special_tokens=True
    )
    return final_text.strip()

# ── 4. Example Usage ───────────────────────────────────────────
if __name__ == "__main__":
    q = "Compute 47 * 13 minus 5 and also tell me the weather in Paris."
    print("Answer:\n", run_with_multiple_tools(q))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Answer:
 The result of the arithmetic expression 47 * 13 minus 5 is 606. The current temperature in Paris is 35°C.


In [2]:
import json
import re
import random
import requests
from datetime import datetime
import pytz

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# ────────────────────────────────────────────────────────
# 1. Load Granite-3.3-2B-Instruct
# ────────────────────────────────────────────────────────
MODEL_NAME = "ibm-granite/granite-3.3-2b-instruct"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
model.eval()

# ────────────────────────────────────────────────────────
# 2. Define Tools Metadata & Implementations
# ────────────────────────────────────────────────────────
TOOLS = [
    {
        "name": "get_weather",
        "description": "Fetch live weather for a given city using OpenWeatherMap",
        "arguments": {
            "city": {
                "type": "string",
                "description": "City name, e.g., 'Paris, FR'"
            }
        }
    },
    {
        "name": "duckduckgo_search",
        "description": "Run a web search query on DuckDuckGo and return the top results",
        "arguments": {
            "query": {
                "type": "string",
                "description": "Search query string"
            },
            "max_results": {
                "type": "integer",
                "description": "Maximum number of results to return"
            }
        }
    },
    {
        "name": "wiki_summary",
        "description": "Get a short summary for a Wikipedia topic",
        "arguments": {
            "topic": {
                "type": "string",
                "description": "Topic to look up on Wikipedia"
            },
            "sentences": {
                "type": "integer",
                "description": "Number of summary sentences"
            }
        }
    },
    {
        "name": "get_current_time",
        "description": "Return the current date and time, optionally for a specified timezone",
        "arguments": {
            "timezone": {
                "type": "string",
                "description": "IANA timezone string, e.g., 'Europe/Paris' (if omitted, use local time)"
            }
        }
    }
]

# — Real Weather Tool —────────────────────────────────────────
OPENWEATHER_API_KEY = "YOUR_OPENWEATHERMAP_API_KEY"

def get_weather(city: str) -> str:
    """Fetch current weather from OpenWeatherMap API."""
    url = (
        f"http://api.openweathermap.org/data/2.5/weather"
        f"?q={city}&appid={OPENWEATHER_API_KEY}&units=metric"
    )
    res = requests.get(url)
    if res.status_code != 200:
        return f"Error fetching weather: {res.text}"
    data = res.json()
    desc = data["weather"][0]["description"]
    temp = data["main"]["temp"]
    return f"Weather in {city}: {temp}°C, {desc}."  # :contentReference[oaicite:4]{index=4}

# — DuckDuckGo Search Tool —────────────────────────────────────
from duckduckgo_search import ddg

def duckduckgo_search_tool(query: str, max_results: int = 5) -> str:
    """Perform a DuckDuckGo search and return titles and URLs."""
    results = ddg(query, max_results=max_results)
    if not results:
        return "No results found."
    output = []
    for r in results:
        title = r.get("title", "No title")
        href  = r.get("href", "")
        output.append(f"- {title}: {href}")
    return "\n".join(output)  # :contentReference[oaicite:5]{index=5}

# — Wikipedia Summary Tool —────────────────────────────────────
import wikipedia

def wiki_summary(topic: str, sentences: int = 2) -> str:
    """Get a summary from Wikipedia for the given topic."""
    try:
        return wikipedia.summary(topic, sentences=sentences)
    except Exception as e:
        return f"Wikipedia lookup error: {e}"  # :contentReference[oaicite:6]{index=6}

# — Current Time Tool —─────────────────────────────────────────
def get_current_time(timezone: str = None) -> str:
    """Return current date & time in local zone or specified timezone."""
    if timezone:
        try:
            tz = pytz.timezone(timezone)
        except Exception:
            return f"Invalid timezone: {timezone}"
        now = datetime.now(tz)
    else:
        now = datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S %Z")  # :contentReference[oaicite:7]{index=7}

# ────────────────────────────────────────────────────────
# 3. Tool-Calling Loop Function (handles multiple calls)
# ────────────────────────────────────────────────────────
def run_agent_with_tools(question: str) -> str:
    conv = [{"role": "user", "content": question}]
    # First pass: model may emit multiple tool calls
    inputs = tokenizer.apply_chat_template(
        conv,
        tools=TOOLS,
        thinking=True,
        return_tensors="pt",
        return_dict=True,
        add_generation_prompt=True
    ).to(DEVICE)

    out = model.generate(**inputs, max_new_tokens=512)
    chunk = tokenizer.decode(
        out[0, inputs["input_ids"].shape[1]:],
        skip_special_tokens=False
    )

    # Extract JSON array of tool calls
    match = re.search(r"<\|tool_call\|>(\[\{.*?\}\])", chunk, re.DOTALL)
    if not match:
        return chunk.strip()

    calls = json.loads(match.group(1))
    reasoning = chunk.split("<|tool_call|>")[0]
    conv.append({"role": "assistant", "content": reasoning})

    # Execute each tool
    for call in calls:
        name, args = call["name"], call["arguments"]
        if name == "get_weather":
            res = get_weather(args["city"])
        elif name == "duckduckgo_search":
            res = duckduckgo_search_tool(args["query"], args.get("max_results", 5))
        elif name == "wiki_summary":
            res = wiki_summary(args["topic"], args.get("sentences", 2))
        elif name == "get_current_time":
            res = get_current_time(args.get("timezone"))
        else:
            res = f"Unknown tool: {name}"
        conv.append({
            "role": "tool_response",
            "name": name,
            "content": json.dumps({"result": res})
        })

    # Final pass: model integrates tool outputs
    final_inputs = tokenizer.apply_chat_template(
        conv,
        tools=TOOLS,
        thinking=True,
        return_tensors="pt",
        return_dict=True,
        add_generation_prompt=True
    ).to(DEVICE)

    final_out = model.generate(**final_inputs, max_new_tokens=256)
    answer = tokenizer.decode(
        final_out[0, final_inputs["input_ids"].shape[1]:],
        skip_special_tokens=True
    )
    return answer.strip()

# ────────────────────────────────────────────────────────
# 4. Usage Example
# ────────────────────────────────────────────────────────
if __name__ == "__main__":
    query = (
        "What’s the weather in New York City right now, "
        "give me a DuckDuckGo search on 'Python AI tools', "
        "summarize 'Climate change' in three sentences, "
        "and tell me the current time in Tokyo."
    )
    print(run_agent_with_tools(query))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ImportError: cannot import name 'ddg' from 'duckduckgo_search' (/usr/local/lib/python3.11/dist-packages/duckduckgo_search/__init__.py)

In [7]:
pip install transformers torch requests duckduckgo-search wikipedia pytz googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=ac78349de377eb1164594bdab73512dce7c559183185d898a183422744487620
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
  Created wheel for wiki

In [4]:
import json
import re
import requests
from datetime import datetime
import pytz

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from duckduckgo_search import DDGS
import wikipedia
from googletrans import Translator

# ───────────────────────────────────────────────────────────
# 1. Load Granite-3.3-2B-Instruct
# ───────────────────────────────────────────────────────────
MODEL_NAME = "ibm-granite/granite-3.3-2b-instruct"
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model     = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
model.eval()

# ───────────────────────────────────────────────────────────
# 2. Define Tools Metadata & Implementations
# ───────────────────────────────────────────────────────────
TOOLS = [
    {
        "name": "get_weather",
        "description": "Fetch live weather for a city (Open-Meteo free API).",
        "arguments": {
            "city": {"type": "string", "description": "City name, e.g. 'Paris'"}
        }
    },
    {
        "name": "duckduckgo_search",
        "description": "Run a DuckDuckGo search and return top results.",
        "arguments": {
            "query":      {"type": "string", "description": "Search query"},
            "max_results":{"type": "integer","description": "Max results"}
        }
    },
    {
        "name": "wiki_summary",
        "description": "Get a Wikipedia summary for a topic.",
        "arguments": {
            "topic":    {"type": "string", "description": "Topic name"},
            "sentences":{"type": "integer","description": "Number of sentences"}
        }
    },
    {
        "name": "get_current_time",
        "description": "Return current date/time (optional IANA timezone).",
        "arguments": {
            "timezone": {"type": "string", "description": "IANA zone, e.g. 'Asia/Tokyo'"}
        }
    },
    {
        "name": "currency_convert",
        "description": "Convert amount between two currencies (exchangerate.host free).",
        "arguments": {
            "amount":        {"type":"number", "description":"Amount to convert"},
            "from_currency": {"type":"string","description":"Base currency code"},
            "to_currency":   {"type":"string","description":"Target currency code"}
        }
    },
    {
        "name": "translate_text",
        "description": "Translate a text into a target language (googletrans).",
        "arguments": {
            "text":            {"type":"string","description":"Text to translate"},
            "target_language": {"type":"string","description":"Language code, e.g. 'fr'"}
        }
    }
]

# — Tool implementations —
translator = Translator()

def get_weather(city: str) -> str:
    # 1) Geocode via Open-Meteo
    geo = requests.get(
        f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1"
    ).json()
    if not geo.get("results"):
        return f"No location found for '{city}'."
    lat, lon = geo["results"][0]["latitude"], geo["results"][0]["longitude"]

    # 2) Fetch current weather
    w = requests.get(
        f"https://api.open-meteo.com/v1/forecast"
        f"?latitude={lat}&longitude={lon}"
        f"&current_weather=true"
    ).json()
    cw = w.get("current_weather", {})
    return (
        f"Weather in {city}: "
        f"{cw.get('temperature')}°C, wind {cw.get('windspeed')} km/h."
    )

def duckduckgo_search_tool(query: str, max_results: int = 5) -> str:
    """
    Perform a DuckDuckGo search using the DDGS class (no ddg function).
    Returns a bullet list of title + URL.
    """
    with DDGS() as ddgs:                                         # :contentReference[oaicite:0]{index=0}
        results = ddgs.text(query, max_results=max_results)
    if not results:
        return "No results found."
    return "\n".join(f"- {r['title']}: {r['href']}" for r in results)


def wiki_summary(topic: str, sentences: int = 2) -> str:
    try:
        return wikipedia.summary(topic, sentences=sentences)
    except Exception as e:
        return f"Wikipedia error: {e}"

def get_current_time(timezone: str = None) -> str:
    if timezone:
        try:
            now = datetime.now(pytz.timezone(timezone))
        except pytz.UnknownTimeZoneError:
            return f"Invalid timezone '{timezone}'."
    else:
        now = datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S %Z")

def currency_convert(amount: float, from_currency: str, to_currency: str) -> str:
    res = requests.get(
        f"https://api.exchangerate.host/convert"
        f"?from={from_currency}&to={to_currency}&amount={amount}"
    ).json()
    if not res.get("result"):
        return "Conversion failed."
    return f"{amount} {from_currency} = {res['result']} {to_currency}"

def translate_text(text: str, target_language: str) -> str:
    try:
        return translator.translate(text, dest=target_language).text
    except Exception as e:
        return f"Translation error: {e}"

# ───────────────────────────────────────────────────────────
# 3. Tool-Calling Loop (handles multiple calls)
# ───────────────────────────────────────────────────────────
def run_agent_with_tools(question: str) -> str:
    conv = [{"role": "user", "content": question}]

    # Initial pass: model may emit a JSON array of calls
    inputs = tokenizer.apply_chat_template(
        conv, tools=TOOLS, thinking=True,
        return_tensors="pt", return_dict=True,
        add_generation_prompt=True
    ).to(DEVICE)

    raw = model.generate(**inputs, max_new_tokens=512)
    chunk = tokenizer.decode(
        raw[0, inputs["input_ids"].shape[1]:],
        skip_special_tokens=False
    )

    # Extract array of calls
    match = re.search(r"<\|tool_call\|>(\[\{.*?\}\])", chunk, re.DOTALL)
    if not match:
        return chunk.strip()

    calls = json.loads(match.group(1))
    reasoning = chunk.split("<|tool_call|>")[0]
    conv.append({"role": "assistant", "content": reasoning})

    # Execute each tool and append responses
    for call in calls:
        name, args = call["name"], call["arguments"]
        func = globals().get(name + ("_tool" if name=="duckduckgo_search" else ""))
        # special names:
        if name == "duckduckgo_search":
            res = duckduckgo_search_tool(**args)
        else:
            res = globals()[name](**args)
        conv.append({
            "role": "tool_response",
            "name": name,
            "content": json.dumps({"result": res})
        })

    # Final pass: integrate tool outputs into answer
    final_inputs = tokenizer.apply_chat_template(
        conv, tools=TOOLS, thinking=True,
        return_tensors="pt", return_dict=True,
        add_generation_prompt=True
    ).to(DEVICE)

    final = model.generate(**final_inputs, max_new_tokens=256)
    return tokenizer.decode(
        final[0, final_inputs["input_ids"].shape[1]:],
        skip_special_tokens=True
    ).strip()

# ───────────────────────────────────────────────────────────
# 4. Example Usage
# ───────────────────────────────────────────────────────────
if __name__ == "__main__":
    q = (
        "Show me today's weather in Berlin, "
        "search DuckDuckGo for 'latest AI breakthroughs', "
        "summarize 'Quantum computing' in 3 sentences, "
        "what time is it in New York, "
        "convert 100 USD to EUR, "
        "and translate 'Hello, how are you?' into Spanish."
    )
    print(run_agent_with_tools(q))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Here are the results:

1. The weather in Berlin today is 15.6°C with a wind speed of 15.1 km/h.
2. Here are some of the latest AI breakthroughs:
   - 6 Game-Changing AI Breakthroughs That Defined 2024 - Forbes
   - Artificial Intelligence News -- ScienceDaily
   - AI for everything: 10 Breakthrough Technologies 2024 - Technology Review
   - AI Innovations 2025: The Latest Breakthroughs Shaping the Future - Front Media
   - Artificial intelligence | MIT News | Massachusetts Institute of Technology
3. A quantum computer is a computer that exploits quantum mechanical phenomena, using specialized hardware to perform calculations exponentially faster than classical computers for certain tasks.
4. The current time in New York is 2025-04-25 13:23:23 EDT.
5. The conversion from 100 USD to EUR failed.
6. The translation of "Hello, how are you?" into Spanish is "Hola, ¿cómo estás?"


In [7]:
import json
import re
import requests
from datetime import datetime
import pytz

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from duckduckgo_search import DDGS
import wikipedia
from googletrans import Translator

# ───────────────────────────────────────────────────────────
# 1. Load Granite-3.3-2B-Instruct
# ───────────────────────────────────────────────────────────
MODEL_NAME = "ibm-granite/granite-3.3-2b-instruct"
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model     = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
model.eval()

# ───────────────────────────────────────────────────────────
# 2. Define Tools Metadata & Implementations
# ───────────────────────────────────────────────────────────
TOOLS = [
    {
        "name": "get_weather",
        "description": "Fetch live weather for a city (Open-Meteo free API)",
        "arguments": {
            "city": {"type": "string", "description": "City name, e.g. 'Paris'"}
        }
    },
    {
        "name": "duckduckgo_search",
        "description": "Run a DuckDuckGo search and return top results",
        "arguments": {
            "query":      {"type": "string",  "description": "Search query"},
            "max_results":{"type": "integer", "description": "Max results"}
        }
    },
    {
        "name": "wiki_summary",
        "description": "Get a Wikipedia summary for a topic",
        "arguments": {
            "topic":    {"type": "string", "description": "Topic name"},
            "sentences":{"type": "integer","description": "Number of sentences"}
        }
    },
    {
        "name": "get_current_time",
        "description": "Return current date/time (optional IANA timezone)",
        "arguments": {
            "timezone": {"type": "string", "description": "IANA zone, e.g. 'Asia/Tokyo'"}
        }
    },
    {
        "name": "currency_open_er_api",
        "description": "Convert amount using open.er-api.com (no key)",
        "arguments": {
            "amount":       {"type": "number", "description": "Amount to convert"},
            "from_currency":{"type":"string","description":"Base currency code"},
            "to_currency":  {"type":"string","description":"Target currency code"}
        }
    },
    {
        "name": "currency_frankfurter",
        "description": "Convert amount using Frankfurter API (no key)",
        "arguments": {
            "amount":       {"type": "number", "description": "Amount to convert"},
            "from_currency":{"type":"string","description":"Base currency code"},
            "to_currency":  {"type":"string","description":"Target currency code"}
        }
    },
    {
        "name": "translate_text",
        "description": "Translate a text into a target language (googletrans)",
        "arguments": {
            "text":            {"type":"string","description":"Text to translate"},
            "target_language": {"type":"string","description":"Language code, e.g. 'fr'"}
        }
    }
]

translator = Translator()

def get_weather(city: str) -> str:
    # 1) Geocode via Open-Meteo
    geo = requests.get(
        f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1"
    ).json()
    if not geo.get("results"):
        return f"Weather lookup failed for '{city}'."
    lat, lon = geo["results"][0]["latitude"], geo["results"][0]["longitude"]
    # 2) Fetch current weather
    w = requests.get(
        f"https://api.open-meteo.com/v1/forecast"
        f"?latitude={lat}&longitude={lon}"
        f"&current_weather=true"
    ).json()
    cw = w.get("current_weather", {})
    return (
        f"Weather in {city}: "
        f"{cw.get('temperature')}°C, wind {cw.get('windspeed')} km/h."
    )

def duckduckgo_search_tool(query: str, max_results: int = 5) -> str:
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=max_results)
    if not results:
        return "DuckDuckGo search returned no results."
    return "\n".join(f"- {r['title']}: {r['href']}" for r in results)

def wiki_summary(topic: str, sentences: int = 2) -> str:
    try:
        return wikipedia.summary(topic, sentences=sentences)
    except Exception as e:
        return f"Wikipedia lookup failed: {e}"

def get_current_time(timezone: str = None) -> str:
    try:
        now = datetime.now(pytz.timezone(timezone)) if timezone else datetime.now()
    except Exception:
        return f"Time lookup failed for timezone '{timezone}'."
    return now.strftime("%Y-%m-%d %H:%M:%S %Z")

def currency_open_er_api(amount: float, from_currency: str, to_currency: str) -> str:
    r = requests.get(f"https://open.er-api.com/v6/latest/{from_currency}")
    data = r.json()
    rate = data.get("rates", {}).get(to_currency)
    if rate is None:
        return "Currency conversion failed."
    return f"{amount} {from_currency} = {amount * rate:.2f} {to_currency}"

def currency_frankfurter(amount: float, from_currency: str, to_currency: str) -> str:
    url = (
        "https://api.frankfurter.app/latest"
        f"?amount={amount}&from={from_currency}&to={to_currency}"
    )
    data = requests.get(url).json()
    rate = data.get("rates", {}).get(to_currency)
    if rate is None:
        return "Currency conversion failed."
    return f"{amount} {from_currency} = {rate:.2f} {to_currency}"

def translate_text(text: str, target_language: str) -> str:
    try:
        return translator.translate(text, dest=target_language).text
    except Exception as e:
        return f"Translation failed: {e}"

# ───────────────────────────────────────────────────────────
# 3. Tool-Calling Loop with Fallback
# ───────────────────────────────────────────────────────────
def run_agent_with_tools_with_fallback(question: str) -> str:
    conv = [{"role": "user", "content": question}]

    # Initial pass: model may emit JSON array of calls
    inputs = tokenizer.apply_chat_template(
        conv, tools=TOOLS, thinking=True,
        return_tensors="pt", return_dict=True,
        add_generation_prompt=True
    ).to(DEVICE)

    raw   = model.generate(**inputs, max_new_tokens=512)
    chunk = tokenizer.decode(
        raw[0, inputs["input_ids"].shape[1]:],
        skip_special_tokens=False
    )

    # Extract calls array
    match = re.search(r"<\|tool_call\|>(\[\{.*?\}\])", chunk, re.DOTALL)
    if not match:
        return chunk.strip()

    calls = json.loads(match.group(1))
    reasoning = chunk.split("<|tool_call|>")[0]
    conv.append({"role": "assistant", "content": reasoning})

    # Execute each tool, append responses & fallback if needed
    for call in calls:
        name, args = call["name"], call["arguments"]
        func = globals().get(name)
        res = func(**args) if func else f"Unknown tool: {name}"

        conv.append({
            "role": "tool_response",
            "name": name,
            "content": json.dumps({"result": res})
        })
        if "failed" in res.lower() or "error" in res.lower():
            conv.append({
                "role": "assistant",
                "content": (
                    f"The `{name}` tool failed. "
                    "Please try to compute or estimate the answer yourself; "
                    "otherwise admit the failure."
                )
            })

    # Final pass: integrate all tool outputs
    final_inputs = tokenizer.apply_chat_template(
        conv, tools=TOOLS, thinking=True,
        return_tensors="pt", return_dict=True,
        add_generation_prompt=True
    ).to(DEVICE)

    final = model.generate(**final_inputs, max_new_tokens=256)
    return tokenizer.decode(
        final[0, final_inputs["input_ids"].shape[1]:],
        skip_special_tokens=True
    ).strip()

# ───────────────────────────────────────────────────────────
# 4. Example Usage
# ───────────────────────────────────────────────────────────
if __name__ == "__main__":
    query = (
        "Show me today's weather in Berlin, "
        "search DuckDuckGo for 'latest AI breakthroughs', "
        "summarize 'Quantum computing' in 3 sentences, "
        "what time is it in Pune , Maharastra, "
        "convert 100 USD to EUR using both free APIs, "
        "and translate 'Hello, how are you?' into Spanish."
        "execute a python code using execute tool"
    )
    print(run_agent_with_tools_with_fallback(query))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

1. Today's weather in Berlin is 15.2°C with a wind speed of 14.7 km/h.
2. According to DuckDuckGo, the latest AI breakthroughs include advancements in machine learning, natural language processing, and computer vision.
3. Quantum computing is a field of computing that leverages quantum mechanics to process information. It has the potential to solve complex problems much faster than classical computers.
4. The current time in Pune, Maharashtra is 23:06 on April 25, 2025 (IST).
5. The conversion of 100 USD to EUR using the free open.er-api.com API is 87.91 EUR.
6. The translation of "Hello, how are you?" into Spanish is "Hola, ¿cómo estás?".


In [9]:
import json
import re
import requests
from datetime import datetime
import pytz

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from duckduckgo_search import DDGS
import wikipedia
from googletrans import Translator

# ───────────────────────────────────────────────────────────
# 1. Load Granite-3.3-2B-Instruct
# ───────────────────────────────────────────────────────────
MODEL_NAME = "ibm-granite/granite-3.3-2b-instruct"
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model     = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
model.eval()

# ───────────────────────────────────────────────────────────
# 2. Define Tools Metadata & Implementations
# ───────────────────────────────────────────────────────────
TOOLS = [
    {
        "name": "get_weather",
        "description": "Fetch live weather for a city (Open-Meteo free API)",
        "arguments": {
            "city": {"type": "string", "description": "City name, e.g. 'Paris'"}
        }
    },
    {
        "name": "duckduckgo_search",
        "description": "Run a DuckDuckGo search and return top results",
        "arguments": {
            "query":      {"type": "string",  "description": "Search query"},
            "max_results":{"type": "integer", "description": "Max results"}
        }
    },
    {
        "name": "wiki_summary",
        "description": "Get a Wikipedia summary for a topic",
        "arguments": {
            "topic":    {"type": "string", "description": "Topic name"},
            "sentences":{"type": "integer","description": "Number of sentences"}
        }
    },
    {
        "name": "get_current_time",
        "description": "Return current date/time (optional IANA timezone)",
        "arguments": {
            "timezone": {"type": "string", "description": "IANA zone, e.g. 'Asia/Tokyo'"}
        }
    },
    {
        "name": "currency_open_er_api",
        "description": "Convert amount using open.er-api.com (no key)",
        "arguments": {
            "amount":       {"type": "number", "description": "Amount to convert"},
            "from_currency":{"type":"string","description":"Base currency code"},
            "to_currency":  {"type":"string","description":"Target currency code"}
        }
    },
    {
        "name": "currency_frankfurter",
        "description": "Convert amount using Frankfurter API (no key)",
        "arguments": {
            "amount":       {"type": "number", "description": "Amount to convert"},
            "from_currency":{"type":"string","description":"Base currency code"},
            "to_currency":  {"type":"string","description":"Target currency code"}
        }
    },
    {
        "name": "translate_text",
        "description": "Translate a text into a target language (googletrans)",
        "arguments": {
            "text":            {"type":"string","description":"Text to translate"},
            "target_language": {"type":"string","description":"Language code, e.g. 'fr'"}
        }
    }
]

translator = Translator()

def get_weather(city: str) -> str:
    # 1) Geocode via Open-Meteo
    geo = requests.get(
        f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1"
    ).json()
    if not geo.get("results"):
        return f"Weather lookup failed for '{city}'."
    lat, lon = geo["results"][0]["latitude"], geo["results"][0]["longitude"]
    # 2) Fetch current weather
    w = requests.get(
        f"https://api.open-meteo.com/v1/forecast"
        f"?latitude={lat}&longitude={lon}"
        f"&current_weather=true"
    ).json()
    cw = w.get("current_weather", {})
    return (
        f"Weather in {city}: "
        f"{cw.get('temperature')}°C, wind {cw.get('windspeed')} km/h."
    )

def duckduckgo_search_tool(query: str, max_results: int = 5) -> str:
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=max_results)
    if not results:
        return "DuckDuckGo search returned no results."
    return "\n".join(f"- {r['title']}: {r['href']}" for r in results)

def wiki_summary(topic: str, sentences: int = 2) -> str:
    try:
        return wikipedia.summary(topic, sentences=sentences)
    except Exception as e:
        return f"Wikipedia lookup failed: {e}"

def get_current_time(timezone: str = None) -> str:
    try:
        now = datetime.now(pytz.timezone(timezone)) if timezone else datetime.now()
    except Exception:
        return f"Time lookup failed for timezone '{timezone}'."
    return now.strftime("%Y-%m-%d %H:%M:%S %Z")

def currency_open_er_api(amount: float, from_currency: str, to_currency: str) -> str:
    r = requests.get(f"https://open.er-api.com/v6/latest/{from_currency}")
    data = r.json()
    rate = data.get("rates", {}).get(to_currency)
    if rate is None:
        return "Currency conversion failed."
    return f"{amount} {from_currency} = {amount * rate:.2f} {to_currency}"

def currency_frankfurter(amount: float, from_currency: str, to_currency: str) -> str:
    url = (
        "https://api.frankfurter.app/latest"
        f"?amount={amount}&from={from_currency}&to={to_currency}"
    )
    data = requests.get(url).json()
    rate = data.get("rates", {}).get(to_currency)
    if rate is None:
        return "Currency conversion failed."
    return f"{amount} {from_currency} = {rate:.2f} {to_currency}"

def translate_text(text: str, target_language: str) -> str:
    try:
        return translator.translate(text, dest=target_language).text
    except Exception as e:
        return f"Translation failed: {e}"

# ───────────────────────────────────────────────────────────
# 3. Tool-Calling Loop with Fallback
# ───────────────────────────────────────────────────────────
def run_agent_with_tools_with_fallback(question: str) -> str:
    conv = [{"role": "user", "content": question}]

    # Initial pass: model may emit JSON array of calls
    inputs = tokenizer.apply_chat_template(
        conv, tools=TOOLS, thinking=True,
        return_tensors="pt", return_dict=True,
        add_generation_prompt=True
    ).to(DEVICE)

    raw   = model.generate(**inputs, max_new_tokens=512)
    chunk = tokenizer.decode(
        raw[0, inputs["input_ids"].shape[1]:],
        skip_special_tokens=False
    )

    # Extract calls array
    match = re.search(r"<\|tool_call\|>(\[\{.*?\}\])", chunk, re.DOTALL)
    if not match:
        return chunk.strip()

    calls = json.loads(match.group(1))
    reasoning = chunk.split("<|tool_call|>")[0]
    conv.append({"role": "assistant", "content": reasoning})

    # Execute each tool, append responses & fallback if needed
    for call in calls:
        name, args = call["name"], call["arguments"]
        func = globals().get(name)
        if func:
            try:
                res = func(**args)
            except Exception as e:
                res = f"Execution error in `{name}`: {e}"
        else:
            res = f"Tool `{name}` is not available. Please try to compute or estimate the answer yourself."

        conv.append({
            "role": "tool_response",
            "name": name,
            "content": json.dumps({"result": res})
        })

        # If tool execution failed or tool is missing, prompt the model to handle it
        if "failed" in res.lower() or "error" in res.lower() or "not available" in res.lower():
            conv.append({
                "role": "assistant",
                "content": (
                    f"The `{name}` tool could not be used. "
                    "Please attempt to process the request using your internal knowledge."
                )
            })

    # Final pass: integrate all tool outputs
    final_inputs = tokenizer.apply_chat_template(
        conv, tools=TOOLS, thinking=True,
        return_tensors="pt", return_dict=True,
        add_generation_prompt=True
    ).to(DEVICE)

    final = model.generate(**final_inputs, max_new_tokens=9000)
    return tokenizer.decode(
        final[0, final_inputs["input_ids"].shape[1]:],
        skip_special_tokens=True
    ).strip()


# ───────────────────────────────────────────────────────────
# 4. Example Usage
# ───────────────────────────────────────────────────────────
if __name__ == "__main__":
    query = (
        "Show me today's weather in Berlin, "
        "search DuckDuckGo for 'latest AI breakthroughs', "
        "summarize 'Quantum computing' in 3 sentences, "
        "what time is it in Pune , Maharastra, "
        "convert 100 USD to EUR using both free APIs, "
        "and translate 'Hello, how are you?' into Spanish."
        "execute a python code using execute tool"
    )
    print(run_agent_with_tools_with_fallback(query))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The translation of "Hello, how are you?" into Spanish is "\u00bfHola, c\u00f3mo est\u00e1s?".


## all tool

In [ ]:
import json
import re
import requests
import sys
import io
from datetime import datetime
import pytz

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from duckduckgo_search import DDGS
import wikipedia
from googletrans import Translator

# ───────────────────────────────────────────────────────────
# 1. Load Granite-3.3-2B-Instruct
# ───────────────────────────────────────────────────────────
MODEL_NAME = "ibm-granite/granite-3.3-2b-instruct"
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model     = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
model.eval()

# ───────────────────────────────────────────────────────────
# 2. Define Tools Metadata & Implementations
# ───────────────────────────────────────────────────────────
TOOLS = [
    {
        "name": "get_weather",
        "description": "Fetch live weather for a city (Open-Meteo free API)",
        "arguments": {
            "city": {"type": "string", "description": "City name, e.g. 'Paris'"}
        }
    },
    {
        "name": "duckduckgo_search",
        "description": "Run a DuckDuckGo search and return top results",
        "arguments": {
            "query":      {"type": "string",  "description": "Search query"},
            "max_results":{"type": "integer", "description": "Max results"}
        }
    },
    {
        "name": "wiki_summary",
        "description": "Get a Wikipedia summary for a topic",
        "arguments": {
            "topic":    {"type": "string", "description": "Topic name"},
            "sentences":{"type": "integer","description": "Number of sentences"}
        }
    },
    {
        "name": "get_current_time",
        "description": "Return current date/time (optional IANA timezone)",
        "arguments": {
            "timezone": {"type": "string", "description": "IANA zone, e.g. 'Asia/Tokyo'"}
        }
    },
    {
        "name": "currency_open_er_api",
        "description": "Convert amount using open.er-api.com (no key)",
        "arguments": {
            "amount":       {"type": "number", "description": "Amount to convert"},
            "from_currency":{"type":"string","description":"Base currency code"},
            "to_currency":  {"type":"string","description":"Target currency code"}
        }
    },
    {
        "name": "currency_frankfurter",
        "description": "Convert amount using Frankfurter API (no key)",
        "arguments": {
            "amount":       {"type": "number", "description": "Amount to convert"},
            "from_currency":{"type":"string","description":"Base currency code"},
            "to_currency":  {"type":"string","description":"Target currency code"}
        }
    },
    {
        "name": "translate_text",
        "description": "Translate a text into a target language (googletrans)",
        "arguments": {
            "text":            {"type":"string","description":"Text to translate"},
            "target_language": {"type":"string","description":"Language code, e.g. 'fr'"}
        }
    },
    {
        "name": "execute_python",
        "description": "Execute a Python code snippet and return its output or error",
        "arguments": {
            "code": {"type":"string", "description":"The Python code to run"}
        }
    }
]

translator = Translator()

# — Tool implementations —
def get_weather(city: str) -> str:
    geo = requests.get(
        f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1"
    ).json()
    if not geo.get("results"):
        return f"Weather lookup failed for '{city}'."
    lat, lon = geo["results"][0]["latitude"], geo["results"][0]["longitude"]
    w = requests.get(
        f"https://api.open-meteo.com/v1/forecast"
        f"?latitude={lat}&longitude={lon}"
        f"&current_weather=true"
    ).json()
    cw = w.get("current_weather", {})
    return (
        f"Weather in {city}: "
        f"{cw.get('temperature')}°C, wind {cw.get('windspeed')} km/h."
    )

def duckduckgo_search_tool(query: str, max_results: int = 5) -> str:
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=max_results)
    if not results:
        return "DuckDuckGo search returned no results."
    return "\n".join(f"- {r['title']}: {r['href']}" for r in results)

def wiki_summary(topic: str, sentences: int = 2) -> str:
    try:
        return wikipedia.summary(topic, sentences=sentences)
    except Exception as e:
        return f"Wikipedia lookup failed: {e}"

def get_current_time(timezone: str = None) -> str:
    try:
        now = datetime.now(pytz.timezone(timezone)) if timezone else datetime.now()
    except Exception:
        return f"Time lookup failed for timezone '{timezone}'."
    return now.strftime("%Y-%m-%d %H:%M:%S %Z")

def currency_open_er_api(amount: float, from_currency: str, to_currency: str) -> str:
    r = requests.get(f"https://open.er-api.com/v6/latest/{from_currency}")
    data = r.json()
    rate = data.get("rates", {}).get(to_currency)
    if rate is None:
        return "Currency conversion failed."
    return f"{amount} {from_currency} = {amount * rate:.2f} {to_currency}"

def currency_frankfurter(amount: float, from_currency: str, to_currency: str) -> str:
    url = (
        "https://api.frankfurter.app/latest"
        f"?amount={amount}&from={from_currency}&to={to_currency}"
    )
    data = requests.get(url).json()
    rate = data.get("rates", {}).get(to_currency)
    if rate is None:
        return "Currency conversion failed."
    return f"{amount} {from_currency} = {rate:.2f} {to_currency}"

def translate_text(text: str, target_language: str) -> str:
    try:
        return translator.translate(text, dest=target_language).text
    except Exception as e:
        return f"Translation failed: {e}"

def execute_python(code: str) -> str:
    """Execute Python code and capture stdout or errors."""
    old_stdout = sys.stdout
    sys.stdout = io.StringIO()
    try:
        exec(code, {})
        output = sys.stdout.getvalue()
        return output or "Code executed successfully with no output."
    except Exception as e:
        return f"Execution error: {e}"
    finally:
        sys.stdout = old_stdout

# ───────────────────────────────────────────────────────────
# 3. Tool-Calling Loop with Fallback
# ───────────────────────────────────────────────────────────
def run_agent_with_tools_with_fallback(question: str) -> str:
    conv = [{"role": "user", "content": question}]

    # Initial pass: model may emit JSON array of calls
    inputs = tokenizer.apply_chat_template(
        conv, tools=TOOLS, thinking=True,
        return_tensors="pt", return_dict=True,
        add_generation_prompt=True
    ).to(DEVICE)

    raw   = model.generate(**inputs, max_new_tokens=512)
    chunk = tokenizer.decode(
        raw[0, inputs["input_ids"].shape[1]:],
        skip_special_tokens=False
    )

    # Extract calls array
    match = re.search(r"<\|tool_call\|>(\[\{.*?\}\])", chunk, re.DOTALL)
    if not match:
        return chunk.strip()

    calls = json.loads(match.group(1))
    reasoning = chunk.split("<|tool_call|>")[0]
    conv.append({"role": "assistant", "content": reasoning})

    # Execute each tool, append responses & fallback if needed
    for call in calls:
        name, args = call["name"], call["arguments"]
        func = globals().get(name)
        res = func(**args) if func else f"Unknown tool: {name}"

        conv.append({
            "role": "tool_response",
            "name": name,
            "content": json.dumps({"result": res})
        })
        if "failed" in res.lower() or "error" in res.lower():
            conv.append({
                "role": "assistant",
                "content": (
                    f"The `{name}` tool failed. "
                    "Please try to compute or estimate the answer yourself; "
                    "otherwise admit the failure."
                )
            })

    # Final pass: integrate all tool outputs
    final_inputs = tokenizer.apply_chat_template(
        conv, tools=TOOLS, thinking=True,
        return_tensors="pt", return_dict=True,
        add_generation_prompt=True
    ).to(DEVICE)

    final = model.generate(**final_inputs, max_new_tokens=256)
    return tokenizer.decode(
        final[0, final_inputs["input_ids"].shape[1]:],
        skip_special_tokens=True
    ).strip()

# ───────────────────────────────────────────────────────────
# 4. Example Usage
# ───────────────────────────────────────────────────────────
if __name__ == "__main__":
    query = (
        "Show me today's weather in Berlin, "
        "search DuckDuckGo for 'latest AI breakthroughs', "
        "summarize 'Quantum computing' in 3 sentences, "
        "what time is it in Pune, Maharashtra, "
        "convert 100 USD to EUR using both free APIs, "
        "translate 'Hello, how are you?' into Spanish, "
        "and execute this Python code using execute tool:\n"
        "for i in range(3): print('Line', i+1)"
    )
    print(run_agent_with_tools_with_fallback(query))
